## 准备数据

In [22]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [23]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [24]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[784, 1000], dtype=tf.float32,initial_value=tf.random.uniform(shape=[784, 1000],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[1,1000], dtype=tf.float32, initial_value=tf.random.uniform(shape=[1, 1000],minval=-0.1, maxval=0.1))
        self.W2 = tf.Variable(shape=[1000, 666], dtype=tf.float32, initial_value=tf.random.uniform(shape=[1000, 666],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[1,666], dtype=tf.float32, initial_value=tf.random.uniform(shape=[1, 666],minval=-0.1, maxval=0.1))
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()
optimizer = optimizers.Adam()

## 计算loss

In [25]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [26]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, tf.constant(train_data[0], dtype=tf.float32), tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, tf.constant(test_data[0], dtype=tf.float32), tf.constant(test_data[1], dtype=tf.int64))
print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 7.002171 ; accuracy 0.00086666667
epoch 1 : loss 6.798895 ; accuracy 0.0018833333
epoch 2 : loss 6.59662 ; accuracy 0.0042
epoch 3 : loss 6.3953958 ; accuracy 0.009333333
epoch 4 : loss 6.1952915 ; accuracy 0.01765
epoch 5 : loss 5.996405 ; accuracy 0.030016666
epoch 6 : loss 5.7988605 ; accuracy 0.047533333
epoch 7 : loss 5.602823 ; accuracy 0.07106667
epoch 8 : loss 5.408492 ; accuracy 0.10081667
epoch 9 : loss 5.2161155 ; accuracy 0.13521667
epoch 10 : loss 5.0259852 ; accuracy 0.17626667
epoch 11 : loss 4.838447 ; accuracy 0.21551667
epoch 12 : loss 4.653892 ; accuracy 0.25418332
epoch 13 : loss 4.472766 ; accuracy 0.28823334
epoch 14 : loss 4.2955503 ; accuracy 0.31906667
epoch 15 : loss 4.122766 ; accuracy 0.3456
epoch 16 : loss 3.9549453 ; accuracy 0.37066665
epoch 17 : loss 3.7926207 ; accuracy 0.39481667
epoch 18 : loss 3.6362975 ; accuracy 0.41756666
epoch 19 : loss 3.4864275 ; accuracy 0.44048333
epoch 20 : loss 3.3433876 ; accuracy 0.46223333
epoch 21 : loss 